In [ ]:
pip install dmba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 28.1 MB/s eta 0:00:00


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import dmba

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from dmba import classificationSummary

from pandas.plotting import parallel_coordinates

from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge

%matplotlib inline

Colab environment detected.


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/EthanRosehart/schulich_data_science/main/GroupProjectDataFINAL.csv')

In [ ]:
df.head()

,Unnamed: 0,QKEY,INTERVIEW_START_W99,INTERVIEW_END_W99,DEVICE_TYPE_W99,LANG_W99,FORM_W99,XTABLET_W99,GENERAL_1,GENERAL_2,...,F_PARTYSUM_FINAL,F_PARTYSUMIDEO_FINAL,F_INC_SDT1,F_REG,F_IDEO,F_INTFREQ,F_VOLSUM,F_INC_TIER2,WEIGHT_W99,political_leaning_index
0,0,100260,2021-11-03 14:25:27,2021-11-03 14:45:34,Laptop/PC,English,Form 2,Non-tablet HH,NaN,1.0,...,Rep/Lean Rep,Conservative Rep/Lean,"$50,000 to less than $60,000",You are ABSOLUTELY CERTAIN that you are regist...,Very conservative,Several times a day,No,Middle income,0.206396,-2
1,1,100314,2021-11-04 12:35:35,2021-11-04 12:55:29,Smartphone,English,Form 1,Non-tablet HH,1.0,NaN,...,Rep/Lean Rep,Moderate/Liberal Rep/Lean,"$40,000 to less than $50,000",You are ABSOLUTELY CERTAIN that you are regist...,Liberal,Several times a day,Yes,Middle income,0.315090,-1
2,2,100363,2021-11-03 20:23:43,2021-11-03 20:36:24,Smartphone,English,Form 1,Non-tablet HH,1.0,NaN,...,Dem/Lean Dem,Moderate/Conservative Dem/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are regist...,Moderate,Several times a day,No,Upper income,0.829579,1
3,3,100598,2021-11-02 13:01:05,2021-11-04 12:37:42,Laptop/PC,English,Form 2,Non-tablet HH,NaN,1.0,...,Rep/Lean Rep,Conservative Rep/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are regist...,Conservative,Several times a day,Yes,Upper income,0.337527,-2
4,4,100637,2021-11-02 12:32:58,2021-11-02 12:46:23,Laptop/PC,English,Form 2,Non-tablet HH,NaN,1.0,...,Rep/Lean Rep,Conservative Rep/Lean,"$30,000 to less than $40,000",You are ABSOLUTELY CERTAIN that you are regist...,Very conservative,Less often,No,Lower income,1.210606,-2


In [ ]:
for col in df.columns:
    print(col)

Unnamed: 0
QKEY
INTERVIEW_START_W99
INTERVIEW_END_W99
DEVICE_TYPE_W99
LANG_W99
FORM_W99
XTABLET_W99
GENERAL_1
GENERAL_2
CNCEXC_W99
GENERAL_4
GENERAL_5
GENERAL_6
GENERAL_7
GENERAL_8
GENERAL_9
GENERAL_9f
GENERAL_10
GENERAL_11
GENERAL_12
GENERAL_13
GENERAL_14
GENERAL_15
GENERAL_16
GENERAL_17
GENERAL_18
GENERAL_19
GENERAL_20
GENERAL_21
CHIP_1
CHIP_2
CHIP_3
CHIP_4_a
CHIP_4_b
CHIP_4_c
CHIP_4_d
CHIP_5
CHIP_6
CHIP_7
CHIP_8
CHIP_9_a
CHIP_9_b
CHIP_9_c
CHIP_10_a
CHIP_10_b
CHIP_10_c
CHIP_10_d
CHIP_11
CHIP_12
CHIP_13_a
CHIP_13_b
CHIP_14_a
CHIP_14_b
CHIP_14_c
CHIP_14_d
GENEV_1
GENEV_2
GENEV_3
GENEV_4_a
GENEV_4_b
GENEV_4_c
GENEV_4_d
GENEV_5
GENEV_6
GENEV_7
GENEV_8
GENEV_9_a
GENEV_9_b
GENEV_9_c
GENEV_9_d
GENEV_10
GENEV_11
GENEV_12_a
GENEV_12_b
GENEV_12_c
GENEV_13_a
GENEV_13_b
EXOV_1
EXOV_2
EXOV_3
EXOV_4_a
EXOV_4_b
EXOV_4_c
EXOV_4_d
EXOV_5
EXOV_6
EXOV_8_a
EXOV_8_b
EXOV_8_c
EXOV_8_d
EXOV_9
EXOV_10
EXOV_11_a
EXOV_11_b
EXOV_12_a
EXOV_12_b
EXOV_12_c
F_RELCOM3CAT
F_METRO
F_CREGION
F_CDIVISION
F_AGECAT
F_GEN

In [ ]:
# General AI Statistics - Generate Percentage of neutral/positive responses

general_cols = [
    'GENERAL_1', 'GENERAL_2', 'CNCEXC_W99', 'GENERAL_4', 'GENERAL_5', 'GENERAL_6', 'GENERAL_7',
    'GENERAL_8', 'GENERAL_9', 'GENERAL_9f', 'GENERAL_10', 'GENERAL_11', 'GENERAL_12', 'GENERAL_13',
    'GENERAL_14', 'GENERAL_15', 'GENERAL_16', 'GENERAL_17', 'GENERAL_18', 'GENERAL_19', 'GENERAL_20',
    'GENERAL_21'
]

# Initialize a dictionary to store counts
counts = {col: {'0': 0, '1': 0} for col in general_cols}

# Calculate the number of 0's and 1's in each column
for col in general_cols:
    counts[col]['0'] = (df[col] == 0).sum()
    counts[col]['1'] = (df[col] == 1).sum()

# Calculate total number of 0's and 1's across all specified columns
total_0 = sum(counts[col]['0'] for col in general_cols)
total_1 = sum(counts[col]['1'] for col in general_cols)
total_responses = total_0 + total_1

# Calculate percentages
percent_0 = (total_0 / total_responses) * 100 if total_responses else 0
percent_1 = (total_1 / total_responses) * 100 if total_responses else 0

# Print the results
print("Counts per column:")
for col in general_cols:
    print(f"{col}: 0's = {counts[col]['0']}, 1's = {counts[col]['1']}")

print("\nTotal counts:")
print(f"Total Negatives: {total_0}")
print(f"Total Positive/Neutral: {total_1}")

print("\nPercentages:")
print(f"Percentage of Negatives: {percent_0:.2f}%")
print(f"Percentage of Positive/Neutrals: {percent_1:.2f}%")






Counts per column:
GENERAL_1: 0's = 482, 1's = 4663
GENERAL_2: 0's = 262, 1's = 4826
CNCEXC_W99: 0's = 3773, 1's = 6450
GENERAL_4: 0's = 3109, 1's = 7071
GENERAL_5: 0's = 1263, 1's = 8922
GENERAL_6: 0's = 3513, 1's = 6671
GENERAL_7: 0's = 3489, 1's = 6689
GENERAL_8: 0's = 2465, 1's = 7721
GENERAL_9: 0's = 1236, 1's = 8971
GENERAL_9f: 0's = 2824, 1's = 7378
GENERAL_10: 0's = 8062, 1's = 2157
GENERAL_11: 0's = 1645, 1's = 8549
GENERAL_12: 0's = 8014, 1's = 2213
GENERAL_13: 0's = 3534, 1's = 6690
GENERAL_14: 0's = 2341, 1's = 7866
GENERAL_15: 0's = 4980, 1's = 5238
GENERAL_16: 0's = 2880, 1's = 7326
GENERAL_17: 0's = 2220, 1's = 7989
GENERAL_18: 0's = 1024, 1's = 9195
GENERAL_19: 0's = 2767, 1's = 7427
GENERAL_20: 0's = 2757, 1's = 7404
GENERAL_21: 0's = 2441, 1's = 7770

Total counts:
Total Negatives: 65081
Total Positive/Neutral: 149186

Percentages:
Percentage of Negatives: 30.37%
Percentage of Positive/Neutrals: 69.63%


In [ ]:
# Brain Chips - Stats

general_cols = [
    "CHIP_1","CHIP_2","CHIP_3","CHIP_4_a","CHIP_4_b","CHIP_4_c","CHIP_4_d","CHIP_5","CHIP_6",
    "CHIP_7","CHIP_8","CHIP_9_a","CHIP_9_b","CHIP_9_c","CHIP_10_a","CHIP_10_b","CHIP_10_c",
    "CHIP_10_d","CHIP_11","CHIP_12","CHIP_13_a","CHIP_13_b","CHIP_14_a","CHIP_14_b","CHIP_14_c","CHIP_14_d"
]

# Initialize a dictionary to store counts
counts = {col: {'0': 0, '1': 0} for col in general_cols}

# Calculate the number of 0's and 1's in each column
for col in general_cols:
    counts[col]['0'] = (df[col] == 0).sum()
    counts[col]['1'] = (df[col] == 1).sum()

# Calculate total number of 0's and 1's across all specified columns
total_0 = sum(counts[col]['0'] for col in general_cols)
total_1 = sum(counts[col]['1'] for col in general_cols)
total_responses = total_0 + total_1

# Calculate percentages
percent_0 = (total_0 / total_responses) * 100 if total_responses else 0
percent_1 = (total_1 / total_responses) * 100 if total_responses else 0

# Print the results
print("Counts per column:")
for col in general_cols:
    print(f"{col}: 0's = {counts[col]['0']}, 1's = {counts[col]['1']}")

print("\nTotal counts:")
print(f"Total Negatives: {total_0}")
print(f"Total Positive/Neutral: {total_1}")

print("\nPercentages:")
print(f"Percentage of Negatives: {percent_0:.2f}%")
print(f"Percentage of Positive/Neutrals: {percent_1:.2f}%")

Counts per column:
CHIP_1: 0's = 3295, 1's = 1797
CHIP_2: 0's = 0, 1's = 0
CHIP_3: 0's = 4045, 1's = 995
CHIP_4_a: 0's = 1713, 1's = 3261
CHIP_4_b: 0's = 852, 1's = 4181
CHIP_4_c: 0's = 1920, 1's = 3053
CHIP_4_d: 0's = 1341, 1's = 3669
CHIP_5: 0's = 3252, 1's = 1779
CHIP_6: 0's = 3146, 1's = 1850
CHIP_7: 0's = 1566, 1's = 3397
CHIP_8: 0's = 484, 1's = 4505
CHIP_9_a: 0's = 323, 1's = 4686
CHIP_9_b: 0's = 350, 1's = 4678
CHIP_9_c: 0's = 273, 1's = 4741
CHIP_10_a: 0's = 1271, 1's = 3752
CHIP_10_b: 0's = 880, 1's = 4144
CHIP_10_c: 0's = 488, 1's = 4532
CHIP_10_d: 0's = 423, 1's = 4604
CHIP_11: 0's = 2706, 1's = 2289
CHIP_12: 0's = 4409, 1's = 559
CHIP_13_a: 0's = 318, 1's = 4705
CHIP_13_b: 0's = 290, 1's = 4732
CHIP_14_a: 0's = 573, 1's = 4485
CHIP_14_b: 0's = 299, 1's = 4763
CHIP_14_c: 0's = 2198, 1's = 2855
CHIP_14_d: 0's = 1753, 1's = 3303

Total counts:
Total Negatives: 38168
Total Positive/Neutral: 87315

Percentages:
Percentage of Negatives: 30.42%
Percentage of Positive/Neutrals: 69

In [ ]:
# Gene Editting Stats

general_cols = [
    "GENEV_1", "GENEV_2", "GENEV_3", "GENEV_4_a", "GENEV_4_b", "GENEV_4_c", "GENEV_4_d",
    "GENEV_5", "GENEV_6", "GENEV_7", "GENEV_8", "GENEV_9_a", "GENEV_9_b", "GENEV_9_c",
    "GENEV_9_d", "GENEV_10", "GENEV_11", "GENEV_12_a", "GENEV_12_b", "GENEV_12_c",
    "GENEV_13_a", "GENEV_13_b"
]

# Initialize a dictionary to store counts
counts = {col: {'0': 0, '1': 0} for col in general_cols}

# Calculate the number of 0's and 1's in each column
for col in general_cols:
    counts[col]['0'] = (df[col] == 0).sum()
    counts[col]['1'] = (df[col] == 1).sum()

# Calculate total number of 0's and 1's across all specified columns
total_0 = sum(counts[col]['0'] for col in general_cols)
total_1 = sum(counts[col]['1'] for col in general_cols)
total_responses = total_0 + total_1

# Calculate percentages
percent_0 = (total_0 / total_responses) * 100 if total_responses else 0
percent_1 = (total_1 / total_responses) * 100 if total_responses else 0

# Print the results
print("Counts per column:")
for col in general_cols:
    print(f"{col}: 0's = {counts[col]['0']}, 1's = {counts[col]['1']}")

print("\nTotal counts:")
print(f"Total Negatives: {total_0}")
print(f"Total Positive/Neutral: {total_1}")

print("\nPercentages:")
print(f"Percentage of Negatives: {percent_0:.2f}%")
print(f"Percentage of Positive/Neutrals: {percent_1:.2f}%")

Counts per column:
GENEV_1: 0's = 2029, 1's = 3062
GENEV_2: 0's = 1515, 1's = 3565
GENEV_3: 0's = 2299, 1's = 2681
GENEV_4_a: 0's = 505, 1's = 4524
GENEV_4_b: 0's = 1430, 1's = 3537
GENEV_4_c: 0's = 1350, 1's = 3627
GENEV_4_d: 0's = 1384, 1's = 3625
GENEV_5: 0's = 2538, 1's = 2473
GENEV_6: 0's = 3945, 1's = 1059
GENEV_7: 0's = 770, 1's = 4184
GENEV_8: 0's = 2914, 1's = 2095
GENEV_9_a: 0's = 1191, 1's = 3816
GENEV_9_b: 0's = 888, 1's = 4127
GENEV_9_c: 0's = 654, 1's = 4354
GENEV_9_d: 0's = 360, 1's = 4651
GENEV_10: 0's = 2661, 1's = 2331
GENEV_11: 0's = 4239, 1's = 770
GENEV_12_a: 0's = 609, 1's = 4374
GENEV_12_b: 0's = 668, 1's = 4325
GENEV_12_c: 0's = 514, 1's = 4487
GENEV_13_a: 0's = 3941, 1's = 1132
GENEV_13_b: 0's = 398, 1's = 4675

Total counts:
Total Negatives: 36802
Total Positive/Neutral: 73474

Percentages:
Percentage of Negatives: 33.37%
Percentage of Positive/Neutrals: 66.63%


In [ ]:
# Exoskeleton Stats

general_cols = [
    "EXOV_1", "EXOV_2", "EXOV_3", "EXOV_4_a", "EXOV_4_b", "EXOV_4_c", "EXOV_4_d",
    "EXOV_5", "EXOV_6", "EXOV_8_a", "EXOV_8_b", "EXOV_8_c", "EXOV_8_d",
    "EXOV_9", "EXOV_10", "EXOV_11_a", "EXOV_11_b", "EXOV_12_a", "EXOV_12_b", "EXOV_12_c"
]

# Initialize a dictionary to store counts
counts = {col: {'0': 0, '1': 0} for col in general_cols}

# Calculate the number of 0's and 1's in each column
for col in general_cols:
    counts[col]['0'] = (df[col] == 0).sum()
    counts[col]['1'] = (df[col] == 1).sum()

# Calculate total number of 0's and 1's across all specified columns
total_0 = sum(counts[col]['0'] for col in general_cols)
total_1 = sum(counts[col]['1'] for col in general_cols)
total_responses = total_0 + total_1

# Calculate percentages
percent_0 = (total_0 / total_responses) * 100 if total_responses else 0
percent_1 = (total_1 / total_responses) * 100 if total_responses else 0

# Print the results
print("Counts per column:")
for col in general_cols:
    print(f"{col}: 0's = {counts[col]['0']}, 1's = {counts[col]['1']}")

print("\nTotal counts:", total_responses)
print(f"Total Negatives: {total_0}")
print(f"Total Positive/Neutral: {total_1}")

print("\nPercentages:")
print(f"Percentage of Negatives: {percent_0:.2f}%")
print(f"Percentage of Positive/Neutrals: {percent_1:.2f}%")

Counts per column:
EXOV_1: 0's = 2881, 1's = 2208
EXOV_2: 0's = 1091, 1's = 3998
EXOV_3: 0's = 2677, 1's = 2377
EXOV_4_a: 0's = 736, 1's = 4313
EXOV_4_b: 0's = 1341, 1's = 3680
EXOV_4_c: 0's = 1326, 1's = 3702
EXOV_4_d: 0's = 1707, 1's = 3320
EXOV_5: 0's = 1650, 1's = 3354
EXOV_6: 0's = 1426, 1's = 3589
EXOV_8_a: 0's = 1181, 1's = 3838
EXOV_8_b: 0's = 515, 1's = 4505
EXOV_8_c: 0's = 370, 1's = 4659
EXOV_8_d: 0's = 455, 1's = 4567
EXOV_9: 0's = 2484, 1's = 2483
EXOV_10: 0's = 3735, 1's = 1267
EXOV_11_a: 0's = 645, 1's = 4380
EXOV_11_b: 0's = 437, 1's = 4598
EXOV_12_a: 0's = 215, 1's = 4848
EXOV_12_b: 0's = 2687, 1's = 2373
EXOV_12_c: 0's = 256, 1's = 4803

Total counts: 100677
Total Negatives: 27815
Total Positive/Neutral: 72862

Percentages:
Percentage of Negatives: 27.63%
Percentage of Positive/Neutrals: 72.37%


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV #GridSearch is for hyperparameter tuning
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, PrecisionRecallDisplay, RocCurveDisplay

In [ ]:
columns = [
    "GENERAL_1", "GENERAL_2", "CNCEXC_W99", "GENERAL_4", "GENERAL_5", "GENERAL_6",
    "GENERAL_7", "GENERAL_8", "GENERAL_9", "GENERAL_9f", "GENERAL_10", "GENERAL_11",
    "GENERAL_12", "GENERAL_13", "GENERAL_14", "GENERAL_15", "GENERAL_16", "GENERAL_17",
    "GENERAL_18", "GENERAL_19", "GENERAL_20", "GENERAL_21", "CHIP_1", "CHIP_2", "CHIP_3",
    "CHIP_4_a", "CHIP_4_b", "CHIP_4_c", "CHIP_4_d", "CHIP_5", "CHIP_6", "CHIP_7",
    "CHIP_8", "CHIP_9_a", "CHIP_9_b", "CHIP_9_c", "CHIP_10_a", "CHIP_10_b", "CHIP_10_c",
    "CHIP_10_d", "CHIP_11", "CHIP_12", "CHIP_13_a", "CHIP_13_b", "CHIP_14_a", "CHIP_14_b",
    "CHIP_14_c", "CHIP_14_d", "GENEV_1", "GENEV_2", "GENEV_3", "GENEV_4_a", "GENEV_4_b",
    "GENEV_4_c", "GENEV_4_d", "GENEV_5", "GENEV_6", "GENEV_7", "GENEV_8", "GENEV_9_a",
    "GENEV_9_b", "GENEV_9_c", "GENEV_9_d", "GENEV_10", "GENEV_11", "GENEV_12_a", "GENEV_12_b",
    "GENEV_12_c", "GENEV_13_a", "GENEV_13_b", "EXOV_1", "EXOV_2", "EXOV_3", "EXOV_4_a",
    "EXOV_4_b", "EXOV_4_c", "EXOV_4_d", "EXOV_5", "EXOV_6", "EXOV_8_a", "EXOV_8_b",
    "EXOV_8_c", "EXOV_8_d", "EXOV_9", "EXOV_10", "EXOV_11_a", "EXOV_11_b", "EXOV_12_a",
    "EXOV_12_b", "EXOV_12_c"
]
# Ensure that all columns are present in the DataFrame
for col in columns:
    if col not in df.columns:
        df[col] = 0  # Add missing columns with default value 0

# Convert all columns to numeric, setting errors='coerce' to handle non-numeric values
df[columns] = df[columns].apply(pd.to_numeric, errors='coerce').fillna(0)


# Calculate the total value of each row
df['Row_Total'] = df[columns].sum(axis=1)

# Display the DataFrame with the total values
print(df['Row_Total'])

0        60.0
1        19.0
2        13.0
3        73.0
4        62.0
         ... 
10255    35.0
10256    55.0
10257    19.0
10258    21.0
10259    10.0
Name: Row_Total, Length: 10260, dtype: float64


In [ ]:
# Sort the DataFrame by Row_Total in descending order
df = df.sort_values(by='Row_Total', ascending=False)

# Calculate the threshold index for the top 70%
threshold_index = int(0.7 * len(df))

# Assign 1 to the top 70% and 0 to the bottom 30%
df['Buy'] = 0
df.iloc[:threshold_index, df.columns.get_loc('Buy')] = 1

# Display the DataFrame with the Buy column
print(df)

       Unnamed: 0          QKEY  INTERVIEW_START_W99    INTERVIEW_END_W99  \
6332         6332  201801175265  2021-11-02 19:05:40  2021-11-02 19:16:05   
10225       10225  202101012819  2021-11-02 13:49:11  2021-11-02 14:08:30   
8110         8110  201901074801  2021-11-02 15:04:06  2021-11-02 15:16:36   
4301         4301  201801093397  2021-11-05 19:00:17  2021-11-05 19:07:47   
691           691        608988  2021-11-04 13:02:13  2021-11-04 13:19:24   
...           ...           ...                  ...                  ...   
4142         4142  201801087452  2021-11-02 17:54:32  2021-11-02 18:05:15   
3362         3362  201801048996  2021-11-02 14:12:10  2021-11-02 14:23:39   
6823         6823  201901002110  2021-11-02 12:33:53  2021-11-02 12:56:17   
9086         9086  202001013612  2021-11-02 21:15:14  2021-11-04 12:10:27   
640           640        601608  2021-11-03 18:41:26  2021-11-03 19:09:03   

      DEVICE_TYPE_W99 LANG_W99 FORM_W99    XTABLET_W99  GENERAL_1  GENERAL_

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

# Assuming 'df' is your DataFrame
# Define the input and output columns
input_columns = [
    "F_RELCOM3CAT", "F_METRO", "F_CREGION", "F_CDIVISION", "F_AGECAT", "F_GENDER",
    "F_EDUCCAT", "F_EDUCCAT2", "F_HISP", "F_HISP_ORIGIN", "F_YEARSINUS", "F_RACECMB",
    "F_RACETHNMOD", "F_CITIZEN", "F_BIRTHPLACE", "F_MARITAL", "F_RELIG", "F_BORN",
    "F_RELIGCAT1", "F_ATTEND", "F_RELIMP", "F_PRAY", "F_PARTY_FINAL", "F_PARTYLN_FINAL",
    "F_PARTYSUM_FINAL", "F_PARTYSUMIDEO_FINAL", "F_INC_SDT1", "F_REG", "F_IDEO", "F_INTFREQ",
    "F_VOLSUM", "F_INC_TIER2", "WEIGHT_W99", "political_leaning_index"
]

output_column = "Buy"

# Ensure all necessary columns are in the DataFrame, filling missing ones with default values
for col in input_columns + [output_column]:
    if col not in df.columns:
        df[col] = 0  # Add missing columns with default value 0

# Extract input and output data
X = df[input_columns]
y = df[output_column]

# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.6963937621832359
Classification Report:
              precision    recall  f1-score   support

           0       0.38      0.03      0.05       613
           1       0.70      0.98      0.82      1439

    accuracy                           0.70      2052
   macro avg       0.54      0.50      0.43      2052
weighted avg       0.61      0.70      0.59      2052



In [ ]:
f1_score(y_test,y_pred)

0.8193679327341258

In [ ]:
accuracy_score(y_test,y_pred)

0.6963937621832359

In [ ]:
precision_score(y_test,y_pred)

0.7029850746268657

In [ ]:
recall_score(y_test,y_pred)

0.9819318971507992

In [ ]:
# Logistic Regression Model slaps, I'm done :)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

# Assuming 'df' is your DataFrame
# Define the input and output columns
input_columns = [
    "F_RELCOM3CAT", "F_METRO", "F_CREGION", "F_CDIVISION", "F_AGECAT", "F_GENDER",
    "F_EDUCCAT", "F_EDUCCAT2", "F_HISP", "F_HISP_ORIGIN", "F_YEARSINUS", "F_RACECMB",
    "F_RACETHNMOD", "F_CITIZEN", "F_BIRTHPLACE", "F_MARITAL", "F_RELIG", "F_BORN",
    "F_RELIGCAT1", "F_ATTEND", "F_RELIMP", "F_PRAY", "F_PARTY_FINAL", "F_PARTYLN_FINAL",
    "F_PARTYSUM_FINAL", "F_PARTYSUMIDEO_FINAL", "F_INC_SDT1", "F_REG", "F_IDEO", "F_INTFREQ",
    "F_VOLSUM", "F_INC_TIER2", "WEIGHT_W99", "political_leaning_index"
]

output_column = "Buy"

# Ensure all necessary columns are in the DataFrame, filling missing ones with default values
for col in input_columns + [output_column]:
    if col not in df.columns:
        df[col] = 0  # Add missing columns with default value 0

# Extract input and output data
X = df[input_columns]
y = df[output_column]

# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the logistic regression model
model = LogisticRegression(max_iter=1000)

# Define a reduced hyperparameter grid for faster search
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best Parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy: 0.6988304093567251
Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.02      0.03       613
           1       0.70      0.99      0.82      1439

    accuracy                           0.70      2052
   macro avg       0.56      0.50      0.43      2052
weighted avg       0.61      0.70      0.59      2052



In [ ]:
f1_score(y_test,y_pred)

0.8215935334872978

In [ ]:
accuracy_score(y_test,y_pred)

0.6988304093567251

In [ ]:
precision_score(y_test,y_pred)

0.702716049382716

In [ ]:
recall_score(y_test,y_pred)

0.9888811674774148

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'df' is your DataFrame
# Define the input and output columns
input_columns = [
    "F_RELCOM3CAT", "F_METRO", "F_CREGION", "F_CDIVISION", "F_AGECAT", "F_GENDER",
    "F_EDUCCAT", "F_EDUCCAT2", "F_HISP", "F_HISP_ORIGIN", "F_YEARSINUS", "F_RACECMB",
    "F_RACETHNMOD", "F_CITIZEN", "F_BIRTHPLACE", "F_MARITAL", "F_RELIG", "F_BORN",
    "F_RELIGCAT1", "F_ATTEND", "F_RELIMP", "F_PRAY", "F_PARTY_FINAL", "F_PARTYLN_FINAL",
    "F_PARTYSUM_FINAL", "F_PARTYSUMIDEO_FINAL", "F_INC_SDT1", "F_REG", "F_IDEO", "F_INTFREQ",
    "F_VOLSUM", "F_INC_TIER2", "WEIGHT_W99", "political_leaning_index"
]

output_column = "Buy"

# Ensure all necessary columns are in the DataFrame, filling missing ones with default values
for col in input_columns + [output_column]:
    if col not in df.columns:
        df[col] = 0  # Add missing columns with default value 0

# Extract input and output data
X = df[input_columns]
y = df[output_column]

# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# Convert boolean columns to int
X = X.astype(int)

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Ensure all data is numeric
X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

# Drop any rows with NaN values created by coercion
X = X.dropna()
y = y.loc[X.index]

# Check if there are any remaining object type columns
print("Data types in X:")
print(X.dtypes)

print("\nData types in y:")
print(y.dtypes)

# Ensure no object types are present
if any(X.dtypes == 'object'):
    raise ValueError("X contains non-numeric data. Check input data.")

if y.dtype == 'object':
    raise ValueError("y contains non-numeric data. Check input data.")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the logistic regression model using statsmodels
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

# Print the summary of the model
print(result.summary())

# Predict on the test set
y_pred_prob = result.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Data types in X:
const                        float64
WEIGHT_W99                     int64
political_leaning_index        int64
F_RELCOM3CAT_High              int64
F_RELCOM3CAT_Low               int64
                              ...   
F_VOLSUM_Refused               int64
F_VOLSUM_Yes                   int64
F_INC_TIER2_Middle income      int64
F_INC_TIER2_Refused            int64
F_INC_TIER2_Upper income       int64
Length: 145, dtype: object

Data types in y:
int64
Optimization terminated successfully.
         Current function value: 0.593251
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                    Buy   No. Observations:                 8208
Model:                          Logit   Df Residuals:                     8078
Method:                           MLE   Df Model:                          129
Date:                Thu, 11 Jul 2024   Pseudo R-squ.:                 0.02942
Time:                       

In [ ]:
# Test Different Inputs

# removed "WEIGHT_W99", - increase of 0.002 f1_score

# removed "F_PARTY_FINAL", "F_PARTYLN_FINAL", "F_PARTYSUM_FINAL", "F_PARTYSUMIDEO_FINAL", - no change

# removed "F_RELCOM3CAT", - increase of 0.002 f1_score

# removed "F_HISP_ORIGIN", "F_HISP",

# removed "F_RELIGCAT1", "F_INTFREQ",

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

# Assuming 'df' is your DataFrame
# Define the input and output columns
input_columns = [
     "F_METRO", "F_CREGION", "F_CDIVISION", "F_AGECAT", "F_GENDER",
    "F_EDUCCAT", "F_EDUCCAT2", "F_YEARSINUS", "F_RACECMB",
    "F_RACETHNMOD", "F_CITIZEN", "F_BIRTHPLACE", "F_MARITAL", "F_RELIG", "F_BORN",
      "F_ATTEND", "F_RELIMP", "F_PRAY",  "F_INC_SDT1", "F_REG", "F_IDEO",
    "F_VOLSUM", "F_INC_TIER2", "political_leaning_index"
]

output_column = "Buy"

# Ensure all necessary columns are in the DataFrame, filling missing ones with default values
for col in input_columns + [output_column]:
    if col not in df.columns:
        df[col] = 0  # Add missing columns with default value 0

# Extract input and output data
X = df[input_columns]
y = df[output_column]

# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the logistic regression model
model = LogisticRegression(max_iter=1000)

# Define a reduced hyperparameter grid for faster search
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)

# Evaluate the best model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best Parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy: 0.7012670565302144
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.01      0.01       613
           1       0.70      1.00      0.82      1439

    accuracy                           0.70      2052
   macro avg       0.60      0.50      0.42      2052
weighted avg       0.64      0.70      0.58      2052



In [ ]:
f1_score(y_test,y_pred)

0.822478386167147

In [ ]:
accuracy_score(y_test,y_pred)

0.7012670565302144

In [ ]:
precision_score(y_test,y_pred)

0.702054794520548

In [ ]:
recall_score(y_test,y_pred)

0.9972202918693537